In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def tileset_info():
    return {
        'min_pos': [-2.5, -2.5],
        'max_pos': [2.5, 2.5],
        'bins_per_dimension': 256,
        'max_width': 5,
        'max_zoom': 50,
        'mirror_tiles': 'false',
    }

In [19]:
from numba import jit, vectorize, guvectorize, float64, complex64, int32, float32
import numpy as np

# from: https://www.ibm.com/developerworks/community/blogs/jfp/entry/How_To_Compute_Mandelbrodt_Set_Quickly?lang=en

@jit(int32(complex64, int32))
def mandelbrot(c,maxiter):
    nreal = 0
    real = 0
    imag = 0
    for n in range(maxiter):
        nreal = real*real - imag*imag + c.real
        imag = 2* real*imag + c.imag
        real = nreal;
        if real * real + imag * imag > 4.0:
            return n
    return 0

@guvectorize([(complex64[:], int32[:], int32[:])], '(n),()->(n)')
def mandelbrot_numpy(c, maxit, output):
    #print("hellow")
    maxiter = maxit[0]
    for i in range(c.shape[0]):
        output[i] = mandelbrot(c[i],maxiter)

"""
import numexpr as ne

def mandelbrot_numpy(c, maxiter):
    output = np.zeros(c.shape)
    z = np.zeros(c.shape, np.complex64)
    for it in range(maxiter):
        notdone = ne.evaluate('z.real*z.real + z.imag*z.imag < 4.0')
        output[notdone] = it
        z = ne.evaluate('where(notdone,z**2+c,z)')
    output[output == maxiter-1] = 0    
    return output
  """      
def mandelbrot_set2(xmin,xmax,ymin,ymax,width=256,height=256,maxiter=500):
    #print("hi")
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
    n3 = mandelbrot_numpy(c,maxiter)
    return n3

#%timeit mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,1024,1024,100)
#%time mandelbrot_set2(-2.5, 2.5, -2.5, 2.5)

CPU times: user 22.9 ms, sys: 1.34 ms, total: 24.2 ms
Wall time: 25 ms
CPU times: user 367 ms, sys: 13.6 ms, total: 381 ms
Wall time: 386 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [20]:
%%time 

import functools as ft
import numpy as np
import hgflask as hgf
import hgflask.client as hgc
import hgtiles.format as hgfo
import hgtiles.utils as hgut
import os.path as op

# from https://plus.maths.org/content/computing-mandelbrot-set

def tiles(tsinfo, z, x, y):
    [from_x, from_y, to_x, to_y] = hgut.tile_bounds(tsinfo, z, x, y)
    tile_size = tsinfo['bins_per_dimension'] if 'bins_per_dimension' in tsinfo else 256
    limit = 4
    
    #print("from_x", from_x, to_x, from_y, to_y)
    #points = mandelbrot(from_x, from_y, to_x, to_y, tile_size)
    points = mandelbrot_set2(from_x, to_x, from_y, to_y)
    #print("points:", points)

    
    # ravel because the points are returned as a 2D array
    #return points.ravel()
    return hgfo.format_dense_tile(points.ravel())
    
tilesets = [{
    'uuid': 'a',
    'handlers': {
        'tileset_info': tileset_info,
        'tiles': ft.partial(hgut.tiles_wrapper_2d, 
                            tiles_function=ft.partial(tiles, tileset_info()))
    }},
    {
        'uuid': 'b',
        'filepath': op.expanduser('~/Dropbox/paper-data/gaia/cooler.mcool')
    }
]

server=hgf.start(tilesets)
#points = tiles(tsinfo, 1, 1, 1)

terminating: NAcWTgoJSCirqSDD0CJ1Uw
sleeping


 * Running on http://0.0.0.0:53320/ (Press CTRL+C to quit)


info: {'a': {'min_pos': [-2.5, -2.5], 'max_pos': [2.5, 2.5], 'bins_per_dimension': 256, 'max_width': 5, 'max_zoom': 50, 'mirror_tiles': 'false'}}


127.0.0.1 - - [28/Sep/2018 10:46:51] "GET /api/v1/tileset_info/?d=a HTTP/1.1" 200 -


ret: 200 b'{\n  "a": {\n    "bins_per_dimension": 256, \n    "max_pos": [\n      2.5, \n      2.5\n    ], \n    "max_width": 5, \n    "max_zoom": 50, \n    "min_pos": [\n      -2.5, \n      -2.5\n    ], \n    "mirror_tiles": "false"\n  }\n}\n'
returning
CPU times: user 14.6 ms, sys: 29.3 ms, total: 43.9 ms
Wall time: 256 ms


...

127.0.0.1 - - [28/Sep/2018 14:11:28] "OPTIONS /api/v1/tiles/?d=a.16.36054.32770&d=a.16.36055.32770&d=a.16.36056.32770&d=a.16.36057.32770&d=a.16.36058.32768&d=a.16.36058.32769&d=a.16.36058.32770&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:11:28] "GET /api/v1/tiles/?d=a.16.36054.32770&d=a.16.36055.32770&d=a.16.36056.32770&d=a.16.36057.32770&d=a.16.36058.32768&d=a.16.36058.32769&d=a.16.36058.32770&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:11:29] "OPTIONS /api/v1/tiles/?d=a.17.72111.65536&d=a.17.72111.65537&d=a.17.72111.65538&d=a.17.72112.65536&d=a.17.72112.65537&d=a.17.72112.65538&d=a.17.72113.65536&d=a.17.72113.65537&d=a.17.72113.65538&d=a.17.72114.65536&d=a.17.72114.65537&d=a.17.72114.65538&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:11:30] "GET /api/v1/tiles/?d=a.17.72111.65536&d=a.17.72111.65537&d=a.17.72111.65538&d=a.17.72112.65536&d=a.17.72112.65537&d=a.17.72112.65538&d=a.17.72113.65536&d=a.17.72113.6

127.0.0.1 - - [28/Sep/2018 14:12:02] "GET /api/v1/tiles/?d=a.20.576896.524295&d=a.20.576897.524295&d=a.20.576898.524295&d=a.20.576899.524295&d=a.20.576900.524295&d=a.20.576901.524295&d=a.20.576901.524296&d=a.20.576901.524297&d=a.20.576901.524298&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:12:06] "OPTIONS /api/v1/tiles/?d=a.18.144223.131073&d=a.18.144223.131074&d=a.18.144223.131075&d=a.18.144224.131073&d=a.18.144224.131074&d=a.18.144224.131075&d=a.18.144225.131073&d=a.18.144225.131074&d=a.18.144225.131075&d=a.18.144226.131073&d=a.18.144226.131074&d=a.18.144226.131075&d=a.18.144227.131073&d=a.18.144227.131074&d=a.18.144227.131075&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:12:07] "GET /api/v1/tiles/?d=a.18.144223.131073&d=a.18.144223.131074&d=a.18.144223.131075&d=a.18.144224.131073&d=a.18.144224.131074&d=a.18.144224.131075&d=a.18.144225.131073&d=a.18.144225.131074&d=a.18.144225.131075&d=a.18.144226.131073&d=a.18.144226.131074&d=a.18.1

127.0.0.1 - - [28/Sep/2018 14:13:21] "GET /api/v1/tiles/?d=a.19.288366.262143&d=a.19.288366.262144&d=a.19.288366.262145&d=a.19.288366.262146&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:13:21] "OPTIONS /api/v1/tiles/?d=a.20.576723.524287&d=a.20.576723.524288&d=a.20.576723.524289&d=a.20.576723.524290&d=a.20.576724.524287&d=a.20.576724.524288&d=a.20.576724.524289&d=a.20.576724.524290&d=a.20.576725.524287&d=a.20.576725.524288&d=a.20.576725.524289&d=a.20.576725.524290&d=a.20.576726.524287&d=a.20.576726.524288&d=a.20.576726.524289&d=a.20.576726.524290&d=a.20.576727.524287&d=a.20.576727.524288&d=a.20.576727.524289&d=a.20.576727.524290&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:13:21] "OPTIONS /api/v1/tiles/?d=a.20.576728.524287&d=a.20.576728.524288&d=a.20.576728.524289&d=a.20.576728.524290&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:13:21] "OPTIONS /api/v1/tiles/?d=a.21.1153448.1048575&d=a.21.1153448.1048576&d=a

127.0.0.1 - - [28/Sep/2018 14:13:37] "OPTIONS /api/v1/tiles/?d=a.17.72089.65535&d=a.17.72089.65536&d=a.17.72089.65537&d=a.17.72089.65534&d=a.17.72090.65534&d=a.17.72091.65534&d=a.17.72092.65534&d=a.17.72093.65534&d=a.17.72094.65534&d=a.17.72094.65535&d=a.17.72094.65536&d=a.17.72094.65537&d=a.16.36044.32767&d=a.16.36044.32768&d=a.16.36044.32769&d=a.16.36045.32767&d=a.16.36045.32768&d=a.16.36045.32769&d=a.16.36046.32767&d=a.16.36046.32768&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:13:37] "OPTIONS /api/v1/tiles/?d=a.16.36046.32769&d=a.16.36047.32767&d=a.16.36047.32768&d=a.16.36047.32769&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:13:37] "OPTIONS /api/v1/tiles/?d=a.16.36048.32767&d=a.16.36048.32768&d=a.16.36048.32769&d=a.16.36043.32767&d=a.16.36043.32768&d=a.16.36043.32769&d=a.16.36043.32766&d=a.16.36044.32766&d=a.16.36045.32766&d=a.16.36046.32766&d=a.16.36047.32766&d=a.16.36048.32766&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 

127.0.0.1 - - [28/Sep/2018 14:14:02] "GET /api/v1/tiles/?d=a.16.36093.32762&d=a.16.36093.32763&d=a.16.36093.32764&d=a.16.36093.32765&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:14:02] "OPTIONS /api/v1/tiles/?d=a.17.72179.65526&d=a.17.72179.65527&d=a.17.72179.65528&d=a.17.72179.65529&d=a.17.72180.65526&d=a.17.72180.65527&d=a.17.72180.65528&d=a.17.72180.65529&d=a.17.72181.65526&d=a.17.72181.65527&d=a.17.72181.65528&d=a.17.72181.65529&d=a.17.72182.65526&d=a.17.72182.65527&d=a.17.72182.65528&d=a.17.72182.65529&d=a.17.72183.65526&d=a.17.72183.65527&d=a.17.72183.65528&d=a.17.72183.65529&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:14:02] "OPTIONS /api/v1/tiles/?d=a.17.72184.65526&d=a.17.72184.65527&d=a.17.72184.65528&d=a.17.72184.65529&s=Uwhjg2O3Q7WeNY6UmXGP8A HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 14:14:03] "GET /api/v1/tiles/?d=a.17.72179.65526&d=a.17.72179.65527&d=a.17.72179.65528&d=a.17.72179.65529&d=a.17.72180.65526&d=a.17.72180.6

In [21]:
#%time tilesets[0]['handlers']['tiles'](['a.1.0.0', 'a.1.0.1'])
server.tiles('a', 0, 0, 0)

{'dense': '...',
 'dtype': 'float16'}

In [22]:
server.process

<Process(Process-3, started)>

In [23]:
import json

conf = hgc.HiGlassConfig()
view = conf.add_view(x=0,y=0, width=6, height=12)
view.add_track('a', 'heatmap', 'center', 
               server=server.api_address,
               height=400, options={
               })
view.add_track(None, 'top-axis', 'top')
view.add_track(None, 'left-axis', 'left')

view2 = conf.add_view(x=6,y=0, width=6, height=12)
view2.add_track('b', 'heatmap', 'center', 
               server=server.api_address)

#print(hgc.to_json_string())
import higlass_jupyter
higlass_jupyter.HiGlassDisplay(viewconf=conf.to_json_string(), 
                               hg_options='{"bounded": false, "renderer": "canvas"}')

HiGlassDisplay(viewconf='{\n  "editable": true,\n  "views": [\n    {\n      "uid": "Lf6Ec1psSt2R75KxbMuHfQ",\n…